# Build "Chat on your own data" logic with RAG pattern using Azure Cognitive Search and Azure OpenAI
This code demonstrates how to use Azure Cognitive Search with OpenAI and Azure Python SDK to query your own data

![Alt text](image-2.png)

How Cognitive search can be used to index your internal knowledge base
![Alt text](image-3.png)


**Azure Cognitive Search has now vector search capabilities** ([Watch this video](https://aka.ms/Vector_SearchSnackableVideo)). The advantages of vector search in Azure Cognitive Search include its integration with other capabilities of Azure Cognitive Search, the ability to use any type of data (text, image, audio, video, etc) from diverse Azure datastores to inform a single generative AI-powered application, and the support of vector fields in the search indexes. It also offers pure vector search, hybrid retrieval, and a sophisticated re-ranking system powered by Bing in a single integrated solution (check the release [blog site](https://techcommunity.microsoft.com/t5/azure-ai-services-blog/announcing-vector-search-in-azure-cognitive-search-public/ba-p/3872868)).


![vector-search](https://techcommunity.microsoft.com/t5/image/serverpage/image-id/489211i001E2B9B34F483C2/image-dimensions/876x416?v=v2)






# Prerequisites
To run the code, install the following packages. Please use the latest pre-release version pip install azure-search-documents --pre.

In [5]:
! pip install azure-search-documents --pre --upgrade
! pip install azure-search --pre --upgrade
! pip install openai

     -------------------------------------- 305.7/305.7 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: azure-search-documents
    Found existing installation: azure-search-documents 11.4.0b6
    Uninstalling azure-search-documents-11.4.0b6:
      Successfully uninstalled azure-search-documents-11.4.0b6



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached azure_search-1.0.0b2-py2.py3-none-any.whl (46 kB)
     -------------------------------------- 192.4/192.4 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.26.3
    Uninstalling azure-core-1.26.3:
      Successfully uninstalled azure-core-1.26.3



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import required libraries and environment variables

In [1]:
# Import required libraries  
import os  
import json  
import openai  
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
     HnswParameters,
    PrioritizedFields,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticSettings,
    SimpleField,
    VectorSearch,    
    HnswVectorSearchAlgorithmConfiguration
)  
  
# Configure environment variables  
load_dotenv("credentials.env")  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
credential = AzureKeyCredential(key)

In [2]:
print(os.getenv("AZURE_SEARCH_INDEX_NAME"))

hotels-vector-index


## Create embeddings
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [37]:
# Generate Document Embeddings using OpenAI Ada 002

# Read the text-sample.json
with open('../data/HotelsData.json', 'r', encoding='utf-8-sig') as file:
    input_data = json.load(file)
    #resultList = list(input_data.items())
print(type(input_data))
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(40))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    print(embeddings)
    return embeddings


# Generate embeddings for title and content fields
for item in input_data:
    print(item)
    title = item['HotelName']
    content = item['Description']
    #print(title)
    title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings
    
    

# Output embeddings to docVectors.json file
with open("../output/hotelsVectors.json", "w") as f:
    json.dump(input_data, f)

<class 'list'>
{'HotelId': '1', 'HotelName': 'Secret Point Hotel', 'Description': "This classic hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.", 'Description_fr': "Cet hôtel classique est idéalement situé sur la principale artère commerciale de la ville en plein cœur de New York. A quelques minutes se trouve la place du temps et le centre historique de la ville, ainsi que d'autres lieux d'intérêt qui font de New York l'une des villes les plus attractives et cosmopolites de l'Amérique.", 'Category': 'Boutique', 'Tags': ['view', 'air conditioning', 'concierge'], 'ParkingIncluded': False, 'LastRenovationDate': '2017-01-18T00:00:00Z', 'Rating': 3.6, 'Address': {'StreetAddress': '677 5th Ave', 'City': 'New York', 'StateProvince': 'NY', 'PostalCode': '10022', 'Cou

## Create your search index
Create your search index schema and vector search configuration:

In [38]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="HotelName", type=SearchFieldDataType.String),
    SearchableField(name="Description", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)




# Create the search index 
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')


 hotels-vector-index created


## Insert text and embeddings into vector store
Add texts and metadata from the JSON data to the vector store:

In [4]:
# Upload some documents to the index
with open('../output/hotelsVectors.json', 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded {len(documents)} documents") 

Uploaded 50 documents


## Perform a vector similarity search

In [7]:
# Pure Vector Search
query = "Travelling with family and my baby, need option in city ,high end rooms need easy to go everywhere option"  
  
search_client = SearchClient(service_endpoint, "hotels-vector-index", credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vectors= [vector],
    select=["HotelName", "Description"],
)  
  
for result in results:  
    print(f"HotelName: {result['HotelName']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Description: {result['Description']}")  
   # print(f"Category: {result['category']}\n")  

[0.019652247428894043, 0.01201648823916912, -0.004713013302534819, -0.00012810326006729156, -0.0002697691379580647, 0.029620779678225517, -0.009086960926651955, -0.011697766371071339, -0.006486326921731234, -0.005492864176630974, -0.00282780802808702, 0.001712281722575426, -0.004848639480769634, 0.005821757949888706, -0.006293059326708317, 0.0037161600776016712, 0.027762699872255325, 0.005187705159187317, 0.015529208816587925, -0.008354579098522663, -0.01951662078499794, -0.002081863349303603, 0.011107792146503925, -0.01946237124502659, -0.0065744840539991856, 0.011874080635607243, 0.014267884194850922, -0.011134917847812176, -0.007873105816543102, -0.0059472122229635715, 0.03938587009906769, 0.006248980760574341, -0.03507295623421669, -0.019801435992121696, -0.002093730727210641, -0.014145821332931519, 0.00487237423658371, -0.0011850346345454454, 0.0221884585916996, -0.007140723522752523, 0.014606949873268604, -0.0010214353678748012, -0.003756847931072116, 0.004065397661179304, -0.001

## Perform a Pure Vector Search with a filter

In [8]:
# Pure Vector Search
query = "Travelling with family and my baby, need option in city ,high end rooms need easy to go everywhere option"  
  
search_client = SearchClient(service_endpoint, "hotels-vector-index", credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vectors= [vector],
    filter="category eq 'Suite'",
    select=["HotelName", "Description"],
)  
  
for result in results:  
    print(f"HotelName: {result['HotelName']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Description: {result['Description']}")  
   # print(f"Category: {result['category']}\n")  

[0.019652247428894043, 0.01201648823916912, -0.004713013302534819, -0.00012810326006729156, -0.0002697691379580647, 0.029620779678225517, -0.009086960926651955, -0.011697766371071339, -0.006486326921731234, -0.005492864176630974, -0.00282780802808702, 0.001712281722575426, -0.004848639480769634, 0.005821757949888706, -0.006293059326708317, 0.0037161600776016712, 0.027762699872255325, 0.005187705159187317, 0.015529208816587925, -0.008354579098522663, -0.01951662078499794, -0.002081863349303603, 0.011107792146503925, -0.01946237124502659, -0.0065744840539991856, 0.011874080635607243, 0.014267884194850922, -0.011134917847812176, -0.007873105816543102, -0.0059472122229635715, 0.03938587009906769, 0.006248980760574341, -0.03507295623421669, -0.019801435992121696, -0.002093730727210641, -0.014145821332931519, 0.00487237423658371, -0.0011850346345454454, 0.0221884585916996, -0.007140723522752523, 0.014606949873268604, -0.0010214353678748012, -0.003756847931072116, 0.004065397661179304, -0.001

## Perform a Hybrid Search : BM25 + Vector search

The accuracy and groudnedness of the final response largely deoends on the quality of the retrieval results. Azure cognitive search provides the capability of hybrid search which has proved to be superior to all the other retrieval techniques.

![Alt text](image-4.png)

In [10]:
# Hybrid Search
query = "Travelling with family and my baby, need option in center ,high end rooms need easy to go everywhere option"  
context=""
  
search_client = SearchClient(service_endpoint, "hotels-vector-index", AzureKeyCredential(key))  
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    select=["HotelName", "Description"],
    top=3
)  
  
for result in results:  
    print(f"HotelName: {result['HotelName']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Description: {result['Description']}")  
   # print(f"Category: {result['category']}\n")      
    context+=result['HotelName']+":"+result['Description']
    context+="\n"


[0.01808234117925167, 0.01838257722556591, -0.0051927026361227036, -0.002279057400301099, 0.0034970566630363464, 0.03433597832918167, -0.011538580991327763, -0.018887517973780632, -0.00205046939663589, 0.006966819521039724, 0.002661174861714244, 0.0074990540742874146, -0.00865222979336977, -0.0006162496283650398, -0.001335705048404634, 0.01412469707429409, 0.024455513805150986, 0.009382347576320171, 0.016362814232707024, -0.008706818334758282, -0.020770810544490814, -0.005752231925725937, 0.007069172337651253, -0.02341833896934986, -0.004674114752560854, 0.011258816346526146, 0.01682681404054165, -0.011715993285179138, 0.005001644138246775, -0.007860701531171799, 0.04124138504266739, 0.0038109389133751392, -0.03125174716114998, -0.024127986282110214, -0.0049197617918252945, -0.011272463947534561, -0.0003686835989356041, 0.0021425869781523943, 0.024728456512093544, -0.009225405752658844, 0.008249642327427864, -0.0006951466202735901, -0.007116937078535557, 0.010665169917047024, -0.003490

In [11]:
print(context)

Inoku:Eco-friendly from our gardens to table, with a rooftop serenity pool and outdoor seating to take in the sunset. Just steps away from the Convention Center. Located in the heart of downtown with modern rooms with stunning city views, 24-7 dining options, free WiFi and easy valet parking.
Old Carrabelle Hotel:Spacious rooms, glamorous suites and residences, rooftop pool, walking access to shopping, dining, entertainment and the city center. Each room comes equipped with a microwave, a coffee maker and a minifridge. In-room entertainment includes complimentary W-Fi and flat-screen TVs. 
Marquis Plaza & Suites:Extend Your Stay. Affordable home away from home, with amenities like free Wi-Fi, full kitchen, and convenient laundry service.



# Next step is to augment your prompts with the retrieved context to ground LLM reponses.

![Alt text](image-5.png)

In [13]:
question="Travelling with family and my baby, need option in center ,high end rooms need easy to go everywhere option"
system_message="You are a helpful assistant.You only use the information in the context to provide answers.If the answer is not in the context, reply with I don't know. context:"+context+"\n"+"Strictly do not provide information which is not in the context."

response = openai.ChatCompletion.create(
                  engine="chat",
                  messages=[
                        {"role": "system", "content": system_message},
                        {"role": "user", "content": "Strictly do not provide information which is not in the context."+question}
                    ],
                  temperature=0.5
                )
# print the response
print(response['choices'][0]['message']['content'])

Based on the information in the context, the Old Carrabelle Hotel would be a good option for you. It offers spacious rooms and glamorous suites, and it is located in the city center with walking access to shopping, dining, and entertainment. Additionally, each room is equipped with a microwave, a coffee maker, and a minifridge, which can be convenient for families with a baby.
